# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.


### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import nltk
import pandas as pd
import pickle
import re
import warnings


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sqlalchemy import create_engine


nltk.download('all')
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\adexs\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\adexs\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\adexs\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\adexs\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\adexs\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [ ]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message'] # feature variable
Y = df.iloc[:, 4:] # target variable

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Remove punctuation, special characters and spaces
    text = re.sub(r'[^a-zA-Z0-9]+', ' ', text)
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    
    # Perform lemmatization on the tokens
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
        
    return lemmas

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.


### 6. Improve your model
Use grid search to find better parameters. 


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [ ]:
def scores(y_test, y_pred):
    '''
    Input
        y_test:
        y_pred
    Output:
        Prints out f1 score, precision, recall and accuracy of the pipeline
    '''
    for i in range(36): # 36 is the number of columns in y_test
        print(f'Feature {i + 1}: {y_test.columns[i]}')
        print(classification_report(y_test.iloc[:, i], y_pred[:, i], zero_division  = 0))
        print(60 * '-')

    # calculate accuracy
    accuracy = (y_pred == y_test.values).mean()
    print(f'Accuracy : {accuracy * 100}')

##### Random Forest Classifier

In [ ]:
pipeline_rfc = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tdidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))  
])
pipeline_rfc

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                ('tdidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [ ]:
# fit pipeline
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                ('tdidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [ ]:
y_pred_rfc = pipeline_rfc.predict(X_test)

In [ ]:
scores(y_test, y_pred_rfc)

Feature 1: related
              precision    recall  f1-score   support

           0       0.68      0.43      0.52      1505
           1       0.84      0.94      0.89      4997
           2       0.40      0.38      0.39        52

    accuracy                           0.81      6554
   macro avg       0.64      0.58      0.60      6554
weighted avg       0.80      0.81      0.80      6554

------------------------------------------------------------
Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5448
           1       0.82      0.51      0.63      1106

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------------
Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
      

In [ ]:
# parameters for Grid Search
pipeline_rfc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
  ('tdidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>),
 'tdidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tdidf__norm': 'l2',
 'tdidf__smooth_idf': True,
 'tdidf__sublinear_tf': False,


In [ ]:
# specify parameters for grid search

parameters_rfc = {
    'tdidf__use_idf' : (True, False),
    # 'clf__estimator__min_samples_split': [2, 3, 4],
    'clf__estimator__n_estimators': [50, 100, 200],
}

In [ ]:
# create grid search object
cv_rfc = GridSearchCV(pipeline_rfc, param_grid= parameters_rfc, cv = 3, n_jobs = 1, verbose = 2)
cv_rfc

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                                       ('tdidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=1,
             param_grid={'clf__estimator__n_estimators': [50, 100, 200],
                         'tdidf__use_idf': (True, False)},
             verbose=2)

In [ ]:
cv_rfc.fit(X_train, y_train)# Fitting using Grid Search

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END clf__estimator__n_estimators=50, tdidf__use_idf=True; total time=10.1min
[CV] END clf__estimator__n_estimators=50, tdidf__use_idf=True; total time=10.5min
[CV] END clf__estimator__n_estimators=50, tdidf__use_idf=True; total time=10.2min
[CV] END clf__estimator__n_estimators=50, tdidf__use_idf=False; total time=10.9min
[CV] END clf__estimator__n_estimators=50, tdidf__use_idf=False; total time=11.0min
[CV] END clf__estimator__n_estimators=50, tdidf__use_idf=False; total time=10.6min
[CV] END clf__estimator__n_estimators=100, tdidf__use_idf=True; total time=20.1min
[CV] END clf__estimator__n_estimators=100, tdidf__use_idf=True; total time=20.9min
[CV] END clf__estimator__n_estimators=100, tdidf__use_idf=True; total time=20.0min
[CV] END clf__estimator__n_estimators=100, tdidf__use_idf=False; total time=20.9min
[CV] END clf__estimator__n_estimators=100, tdidf__use_idf=False; total time=21.9min
[CV] END clf__estimator__n_e

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                                       ('tdidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=1,
             param_grid={'clf__estimator__n_estimators': [50, 100, 200],
                         'tdidf__use_idf': (True, False)},
             verbose=2)

In [ ]:
print(cv_rfc.best_params_)

{'clf__estimator__n_estimators': 100, 'tdidf__use_idf': True}


In [ ]:
# Predicting using the classifier
y_pred = cv_rfc.predict(X_test)
scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.68      0.43      0.53      1505
           1       0.84      0.94      0.89      4997
           2       0.46      0.40      0.43        52

    accuracy                           0.82      6554
   macro avg       0.66      0.59      0.61      6554
weighted avg       0.80      0.82      0.80      6554

------------------------------------------------------------
Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5448
           1       0.82      0.50      0.62      1106

    accuracy                           0.90      6554
   macro avg       0.86      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------------
Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
      

##### KNN

In [ ]:
pipeline_knn = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tdidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(10))) 
])

In [ ]:
# fit the KNN Classifier
pipeline_knn.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                ('tdidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)))])

In [ ]:
y_pred_knn = pipeline_rfc.predict(X_test)

In [ ]:
scores(y_test, y_pred_knn)

Feature 1: related
              precision    recall  f1-score   support

           0       0.68      0.43      0.52      1505
           1       0.84      0.94      0.89      4997
           2       0.40      0.38      0.39        52

    accuracy                           0.81      6554
   macro avg       0.64      0.58      0.60      6554
weighted avg       0.80      0.81      0.80      6554

------------------------------------------------------------
Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5448
           1       0.82      0.51      0.63      1106

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------------
Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
      

In [ ]:
# parameters for Grid Search
pipeline_knn.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
  ('tdidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>),
 'tdidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tdidf__norm': 'l2',
 'tdidf__smooth_idf': True,
 'td

In [ ]:
# specify parameters for grid search

parameters_knn = {
    # 'tdidf__norm': ['l1', 'l2'],
    'tdidf__use_idf' : (True, False),
    'clf__estimator__p': [1, 2]
}

In [ ]:
# create grid search object

cv_knn = GridSearchCV(pipeline_knn, param_grid = parameters_knn, cv = 3, n_jobs = 1, verbose = 2)
cv_knn

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                                       ('tdidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)))]),
             n_jobs=1,
             param_grid={'clf__estimator__p': [1, 2],
                         'tdidf__use_idf': (True, False)},
             verbose=2)

In [ ]:
# fit the classifier
cv_knn.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ...........clf__estimator__p=1, tdidf__use_idf=True; total time= 2.7min
[CV] END ...........clf__estimator__p=1, tdidf__use_idf=True; total time= 2.7min
[CV] END ...........clf__estimator__p=1, tdidf__use_idf=True; total time= 2.7min
[CV] END ..........clf__estimator__p=1, tdidf__use_idf=False; total time= 5.1min
[CV] END ..........clf__estimator__p=1, tdidf__use_idf=False; total time= 5.1min
[CV] END ..........clf__estimator__p=1, tdidf__use_idf=False; total time= 5.1min
[CV] END ...........clf__estimator__p=2, tdidf__use_idf=True; total time= 5.4min
[CV] END ...........clf__estimator__p=2, tdidf__use_idf=True; total time= 5.5min
[CV] END ...........clf__estimator__p=2, tdidf__use_idf=True; total time= 5.5min
[CV] END ..........clf__estimator__p=2, tdidf__use_idf=False; total time= 5.5min
[CV] END ..........clf__estimator__p=2, tdidf__use_idf=False; total time= 5.5min
[CV] END ..........clf__estimator__p=2, tdidf__us

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000294BC627D90>)),
                                       ('tdidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=10)))]),
             n_jobs=1,
             param_grid={'clf__estimator__p': [1, 2],
                         'tdidf__use_idf': (True, False)},
             verbose=2)

In [ ]:
print(cv_knn.best_params_)

{'clf__estimator__p': 2, 'tdidf__use_idf': False}


In [ ]:
# Predicting using the classifier
y_pred = cv_knn.predict(X_test)
scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.59      0.45      0.51      1505
           1       0.84      0.90      0.87      4997
           2       0.36      0.10      0.15        52

    accuracy                           0.79      6554
   macro avg       0.59      0.48      0.51      6554
weighted avg       0.78      0.79      0.78      6554

------------------------------------------------------------
Feature 2: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5448
           1       0.82      0.39      0.53      1106

    accuracy                           0.88      6554
   macro avg       0.85      0.69      0.73      6554
weighted avg       0.88      0.88      0.86      6554

------------------------------------------------------------
Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
file_name = 'classifier.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(cv_rfc, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.